
## White dwarfs

In [ ]:

url = "https://cdsarc.u-strasbg.fr/viz-bin/asu-tsv?-source=J/A+A/420/507/tablea1&-out=Mass&-out=Rad"

In [ ]:

catalog = download(url)

In [ ]:

using CSV
using DataFrames

In [ ]:

df = CSV.read(catalog, DataFrame, comment="#", skipto=4, types=[Float64, Float64],
    delim='\t', ignorerepeated=true, silencewarnings=true);

In [ ]:

dropmissing!(df);

In [ ]:

using PyPlot

In [ ]:

plot(df.Mass, df.Rad, marker="o", linestyle="none",
    fillstyle="none", markersize=4, label="observations")
title("Radius vs. Mass for White Dwarfs")
xlabel("Mass (Solar Masses)")
ylabel("Radius (Solar Radii)")
grid(true)

In [ ]:

using OrdinaryDiffEqTsit5

In [ ]:

"""
    white_dwarf_eqs!(dudr, u, p, r)

The right hand side of the system of 2 dimensionless differential 
equations describing the radial distribution of the density, rho(r),
and mass, m(r), inside a white dwarf star

     m' =  rho r^2
   rho' = -m rho /(gamma(\rho) r^2)

where gamma(rho) = rho^(2/3)/(3 sqrt(1 + rho^(2/3)))
"""
function white_dwarf_eqs!(dudr, u, rho_c, r)
    rho = u[2]
    if rho < 0
        dudr[1] = 0
        dudr[2] = 0
        return
    end
    
    m = u[1]
    r2 = r ^ 2
    dudr[1] = rho * r2
    
    w = rho ^ (2/3)
    gamma = w / (3 * sqrt(1 + w))
    
    if r < 1e-6
        dudr[2] = -rho_c * r * rho / (3 * gamma)
        return
    end
    dudr[2] = -m * rho / (gamma * r2)
end

In [ ]:

rspan = (0.0, 10.0)
condition(u, t, integrator) = u[2]
affect!(integrator) = terminate!(integrator)
cb = ContinuousCallback(condition, affect!);

In [ ]:

rho_c0 = 1
u0_particular = [0.0, rho_c0]


In [ ]:

prob = ODEProblem(white_dwarf_eqs!, u0_particular, rspan, rho_c0)
sol = solve(prob, Tsit5(), callback=cb);

In [ ]:

rad_particular = sol.t
mass_particular = sol[1, :]
rho_particular = sol[2, :]

In [ ]:

plot(rad_particular, rho_particular)
title("Density of White Dwarf")
xlabel("Distance from Center (arb. units)")
ylabel("Density (arb. units)")
grid(true)

$$ \vec{F_g} = -\frac{GMm}{r^2}\hat{r} \implies g = G \frac{M}{r^2} $$

In [ ]:

skipto = 2
plot(rad_particular[skipto:end], mass_particular[skipto:end] .* rad_particular[skipto:end] .^ (-2))
title("Gravity Strength vs. Distance from Center")
xlabel("Distance from Center (arb. units)")
ylabel("Gravity Strength (arb. units)")
grid(true)

In [ ]:

plot(
    rad_particular[skipto:end], 
    mass_particular[skipto:end] .* rho_particular[skipto:end] .* rad_particular[skipto:end] .^ (-2)
)
title("Weight per Unit Volume vs. Distance from Center")
xlabel("Distance from Center (arb. units)")
ylabel("Weight per Unit Volume (arb. units)")
grid(true)

In [ ]:

rhocbegin = 0.08
rhocend = 1e5
np = 100

rhocs = exp.(range(log(rhocbegin), log(rhocend), np));

In [ ]:

radii = zeros(np)
masses = zeros(np);

In [ ]:

for i = 1:np
    rho_c = rhocs[i]
    u0 = [0, rho_c]
    
    prob = ODEProblem(white_dwarf_eqs!, u0, rspan, rho_c)
    sol = solve(prob, Tsit5(), callback=cb);

    radii[i] = sol.t[end]
    masses[i] = sol.u[end][1]
end

# convert to solar radii and solar masses
radii .*= 0.006
masses .*= 0.71;

In [ ]:

plot(masses, radii, label="theory")
plot(df.Mass, df.Rad, marker="o", linestyle="none",
    fillstyle="none", markersize=4, label="observations")
title("Radius vs. Mass for White Dwarfs")
xlabel("Mass (Solar Masses)")
ylabel("Radius (Solar Radii)")
grid(true)
legend()